In [1]:
#importa librerie
import gurobipy as gp
import numpy as np
from gurobipy import GRB

#leggo da file la matrice di copertura 
mod = gp.Model('WDCP')
file = open('TEST1.txt', 'r')
content = file.readlines()
A = []
for i in content:
    array = []
    parse = i.split()
    for j in parse:
        array.append(float(j))
    A.append(array)


numero_righe=len(A)
numero_colonne=len(A[0])


#leggo da file il vettore di punti importani 
file2 = open('TEST1sj.txt', 'r')
content = file2.readline()
pars1=content.split()
s = []
for i in pars1:
    s.append(float(i))

    
A = np.array(A)

#visualizzo la matrice di copertura e il vettore di punti importani
print('A: ',A)
print('righe di A: ',numero_righe)
print('colonne di A: ',numero_colonne)
print('s: ',s)

#messaggio di errore se le lunghezze non coincidono
if (len(s)!=numero_colonne):
    print('errore,le lunghezze non coincidono')
    


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-13
A:  [[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1.]]
righe di A:  4
colonne di A:  16
s:  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]


In [2]:
#parametro alpha per i punti generali
alpha=1.00001
#creazione del modello
mod = gp.Model('WDCP')
L = range(numero_righe)
R = range(numero_colonne)
y = mod.addVars(L,vtype=GRB.BINARY,name="y")
x = mod.addVars(R,vtype=GRB.BINARY,name="x")


# Creazione dell'obiettivo
obj = mod.setObjective(gp.quicksum(y[i] for i in L) - alpha * gp.quicksum((1-s[j])*x[j] for j in R), GRB.MINIMIZE)


#vincoli
for j in R:  
    if s[j]==1: #aggiungo per forza i punti importanti
        mod.addConstr(gp.quicksum(A[i,j]*y[i] for i in L) >= 1)
    else:     #altrimenti, il punto è generale e la sua appartenza alla soluzione dipende da alpha
        mod.addConstr(gp.quicksum(A[i,j] * y[i] for i in L) >= x[j]) 
    #for i in L:
        #mod.addConstr((A[i,j]*y[i])<=x[j])  #se posiziono la telecamera devo segnare i punti coperti

# Risoluzione del problema
mod.optimize()

# Stampa dei risultati

for i in L:
    print(f"y_{i+1} =", int(y[i].x))

for j in R:
    print(f"x_{j+1} =", int(x[j].x))




Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 20 columns and 25 nonzeros
Model fingerprint: 0x2d81bde5
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -1.0000300
Presolve removed 16 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -5.00009 -1.00003 
No other solutions better than -5.00009

Optimal solution found (tolerance 1.00e-04)
Best objective -5.000090000000e+00, best bound -5.000090000000e+00, gap 0.0000%
y_1 = 

In [3]:
puntivisti=0;
    
for j in R:
    puntivisti=puntivisti+int(x[j].x)
    #if (x[j].x==1):
       # print("visti: x",j) 

puntiimportanti=0;
    
for j in range(numero_colonne):
    puntiimportanti=puntiimportanti+s[j]

telecameremesse=0;
for i in L:
    telecameremesse=telecameremesse+int(y[i].x)
    
print("numero di posizioni osservate: ",puntivisti) #punti coperti
print("numero telecamere posizionate: ",telecameremesse) 
print("numero punti importanti: ",puntiimportanti) 



numero di posizioni osservate:  9
numero telecamere posizionate:  4
numero punti importanti:  3.0
